In [21]:
print("SCRIPT IS RUNNING")

import os
import mne
import numpy as np
import pandas as pd
from mne.time_frequency import psd_array_welch

BASE_DIR = r"C:\Users\User\Documents\EEG_Project\rEEG"
SUBJECTS = [f"sub-{i:03d}" for i in range(4, 150)]

def spectral_entropy(psd, freqs, fmin=1, fmax=45, normalize=True):
    idx = np.logical_and(freqs >= fmin, freqs <= fmax)
    psd_band = psd[:, :, idx]
    psd_norm = psd_band / psd_band.sum(axis=-1, keepdims=True)
    se = -np.sum(psd_norm * np.log2(psd_norm + 1e-12), axis=-1)
    if normalize:
        se /= np.log2(psd_norm.shape[-1])
    return se

for SUBJ in SUBJECTS:
    SUBJ_DIR = os.path.join(BASE_DIR, SUBJ)
    subj_num = SUBJ.split('-')[1]  # extract just the number, e.g., "001"
    epo_file = os.path.join(SUBJ_DIR, f"epo_{subj_num}_raw.fif")  # correct filename

    if not os.path.exists(epo_file):
        print("Checking:", epo_file)
        continue
    else:
        print("FOUND:", epo_file)

    # Load epochs
    epochs = mne.read_epochs(epo_file, preload=True, verbose=False)
    sfreq = epochs.info["sfreq"]
    ch_names = epochs.ch_names

    # Compute PSD with 4s epochs and 50% overlap
    n_per_seg = int(sfreq * 4)
    noverlap = n_per_seg // 2

    psds, freqs = psd_array_welch(
        epochs.get_data(), sfreq=sfreq, fmin=1, fmax=45,
        n_fft=1024,
        n_per_seg=n_per_seg,
        n_overlap=noverlap,
        window='hann',
        verbose=False
    )

    # Compute spectral entropy per epoch/channel
    H = spectral_entropy(psds, freqs)

    # Per-epoch/channel CSV (long format)
    df_entropy = pd.DataFrame(H, columns=ch_names)
    df_entropy.insert(0, 'epoch', np.arange(len(df_entropy)))
    df_long = df_entropy.melt(id_vars=['epoch'], var_name='channel', value_name='spec_entropy')
    output_path = os.path.join(SUBJ_DIR, f"{SUBJ}_spectral_entropy.csv")
    df_long.to_csv(output_path, index=False)

    # Mean per subject across all epochs/channels
    subject_mean = H.mean()
    mean_output_path = os.path.join(SUBJ_DIR, f"{SUBJ}_spectral_entropy_mean.csv")
    pd.DataFrame({'subject': [SUBJ], 'mean_spec_entropy': [subject_mean]}).to_csv(mean_output_path, index=False)

    print(f"{SUBJ}: Per-epoch/channel saved, mean spectral entropy = {subject_mean:.4f}")


SCRIPT IS RUNNING
FOUND: C:\Users\User\Documents\EEG_Project\rEEG\sub-004\epo_004_raw.fif


C:\Users\User\AppData\Local\Temp\ipykernel_30704\390887607.py:33: RuntimeWarning: This filename (C:\Users\User\Documents\EEG_Project\rEEG\sub-004\epo_004_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(epo_file, preload=True, verbose=False)


sub-004: Per-epoch/channel saved, mean spectral entropy = 0.7361
FOUND: C:\Users\User\Documents\EEG_Project\rEEG\sub-005\epo_005_raw.fif


C:\Users\User\AppData\Local\Temp\ipykernel_30704\390887607.py:33: RuntimeWarning: This filename (C:\Users\User\Documents\EEG_Project\rEEG\sub-005\epo_005_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(epo_file, preload=True, verbose=False)


sub-005: Per-epoch/channel saved, mean spectral entropy = 0.8390
FOUND: C:\Users\User\Documents\EEG_Project\rEEG\sub-006\epo_006_raw.fif


...

sub-147: Per-epoch/channel saved, mean spectral entropy = 0.8578
FOUND: C:\Users\User\Documents\EEG_Project\rEEG\sub-148\epo_148_raw.fif


C:\Users\User\AppData\Local\Temp\ipykernel_30704\390887607.py:33: RuntimeWarning: This filename (C:\Users\User\Documents\EEG_Project\rEEG\sub-148\epo_148_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(epo_file, preload=True, verbose=False)


sub-148: Per-epoch/channel saved, mean spectral entropy = 0.7185
FOUND: C:\Users\User\Documents\EEG_Project\rEEG\sub-149\epo_149_raw.fif


C:\Users\User\AppData\Local\Temp\ipykernel_30704\390887607.py:33: RuntimeWarning: This filename (C:\Users\User\Documents\EEG_Project\rEEG\sub-149\epo_149_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(epo_file, preload=True, verbose=False)


sub-149: Per-epoch/channel saved, mean spectral entropy = 0.8531


In [22]:
import os
import pandas as pd

BASE_DIR = r"C:\Users\User\Documents\EEG_Project\rEEG"
SUBJECTS = [f"sub-{i:03d}" for i in range(1, 51)]  # first 50 subjects

summary_list = []

for sub in SUBJECTS:
    mean_file = os.path.join(BASE_DIR, sub, f"{sub}_spectral_entropy_mean.csv")
    if os.path.exists(mean_file):
        df = pd.read_csv(mean_file)
        summary_list.append(df)
    else:
        print(f"Missing file: {mean_file}")

# Combine all into one DataFrame
summary_df = pd.concat(summary_list, ignore_index=True)

# Save combined CSV
output_path = os.path.join(BASE_DIR, "mean_spectral_entropy_first50.csv")
summary_df.to_csv(output_path, index=False)
print(f"Saved summary CSV to {output_path}")


Saved summary CSV to C:\Users\User\Documents\EEG_Project\rEEG\mean_spectral_entropy_first50.csv
